<a href="https://colab.research.google.com/github/Pinokcio/ML_Study/blob/main/practice/MNIST_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch
!pip3 install torchvision

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"device : {device}")

device : cuda


In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
train_ds = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y : torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)
test_ds = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y : torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

batch_size = 64
trainDL = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
testDL = DataLoader(test_ds, batch_size = batch_size, shuffle = True)

In [ ]:
import torch.nn as nn

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logit

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------------")
  train(trainDL, model, loss_fn, optimizer)
  test(testDL, model, loss_fn)
print("DONE!")

Epoch 1
-----------------------
loss: 0.658095 [    0/60000]
loss: 0.781283 [ 6400/60000]
loss: 0.611289 [12800/60000]
loss: 0.727196 [19200/60000]
loss: 0.832589 [25600/60000]
loss: 0.750059 [32000/60000]
loss: 0.757918 [38400/60000]
loss: 0.589580 [44800/60000]
loss: 0.742801 [51200/60000]
loss: 0.648321 [57600/60000]
Test Error: 
 Accuracy: 72.2%, Avg loss: 0.742881

Epoch 2
-----------------------
loss: 0.677537 [    0/60000]
loss: 0.742390 [ 6400/60000]
loss: 0.798460 [12800/60000]
loss: 0.772505 [19200/60000]
loss: 0.812432 [25600/60000]
loss: 0.749952 [32000/60000]
loss: 0.680331 [38400/60000]
loss: 0.796026 [44800/60000]
loss: 0.638585 [51200/60000]
loss: 0.570587 [57600/60000]
Test Error: 
 Accuracy: 73.4%, Avg loss: 0.723864

Epoch 3
-----------------------
loss: 0.535791 [    0/60000]
loss: 0.755070 [ 6400/60000]
loss: 0.601112 [12800/60000]
loss: 0.826405 [19200/60000]
loss: 0.565777 [25600/60000]
loss: 0.678926 [32000/60000]
loss: 0.513848 [38400/60000]
loss: 0.663181 [448